# 3.1.1 Tensor

In [1]:
import torch
a = torch.Tensor([[2,3],[4,8],[7,9]])
print('a is {}'.format(a))
print('a size is {}'.format(a.size()))

a is tensor([[2., 3.],
        [4., 8.],
        [7., 9.]])
a size is torch.Size([3, 2])


In [2]:
b = torch.LongTensor([[2,3],[4,8],[7,9]])
print('b is {}'.format(b))

b is tensor([[2, 3],
        [4, 8],
        [7, 9]])


In [3]:
c = torch.zeros((3,2))
print('zero tensor is {}'.format(c))

zero tensor is tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [4]:
d = torch.randn((3,2))
print('zero tensor is {}'.format(d))

zero tensor is tensor([[-1.8671,  0.0204],
        [ 0.6895,  1.7295],
        [ 0.1434, -0.3827]])


## 可以像numpy一样，通过索引取得其中的元素，改变它的值，如：

In [5]:
a[2,0]

tensor(7.)

In [6]:
a[2,0] = 100
print('changed a is {}'.format(a))

changed a is tensor([[  2.,   3.],
        [  4.,   8.],
        [100.,   9.]])


## 在Tensor 与 numpy.ndarray 之间相互转换
a.numpy() ---convert to numpy, __a:tensor__    
torch.__from_numpy(a) --__a:numpy array__

In [7]:
numpyB = b.numpy()
print('conver to numpy is {}'.format(numpyB))

conver to numpy is [[2 3]
 [4 8]
 [7 9]]


In [8]:
import numpy as np
e = np.array([[2,3],[3,4]])
tensorE = torch.from_numpy(e)
print('conver to tensor is {}'.format(tensorE))

conver to tensor is tensor([[2, 3],
        [3, 4]])


## change the type of tensor to float, like a.float()

In [9]:
ftorchE = tensorE.float()
print('float torch is {}'.format(ftorchE))

float torch is tensor([[2., 3.],
        [3., 4.]])


## 通过torch.cuda.is_available()判断，电脑是否支持GPU。

支持的话，如果想__把tensor a放到GPU上__，只需a.cuda()就可以。

In [10]:
torch.cuda.is_available()

True

In [11]:
if torch.cuda.is_available():
    a_cuda = a.cuda()
    print(a_cuda)

tensor([[  2.,   3.],
        [  4.,   8.],
        [100.,   9.]], device='cuda:0')


# 3.1.2 Variable  
Variable 在numpy中不存在，提供了自动求导的功能。   
TensorFlow，神经网络在做运算时，需要先构造一个计算图谱，然后，在里面进行前向传播和反向传播。  

Variable会被放入一个计算图中，然后进行前向传播，反向传播，自动求导。


In [12]:
# Tensor and Variable 没有本质区别，
from torch.autograd import Variable
a = torch.Tensor([[2,3],[4,8]])
va = Variable(a)
va

tensor([[2., 3.],
        [4., 8.]])

## Variable 有三个属性：

1. data：取出Variable里面Tensor的值
2. grad：保存了Variable的反向传播梯度
3. grad_fn：得到Variable的操作

__requires_grad=True，表示对该变量求梯度__

In [13]:
x = Variable(torch.Tensor([1]), requires_grad=True)
w = Variable(torch.Tensor([2]), requires_grad=True)
b = Variable(torch.Tensor([3]), requires_grad=True)
x

tensor([1.], requires_grad=True)

In [14]:
# Build a computational graph.
y = w*x + b

## 1. scalar gradient：标量求导，backward函数的参数可以不写。     
y.backward() is equal to y.backward(torch.FloatTensor([1]))  

## 2. 对矩阵求导，backward函数必须有参数。   
y.backward(torch.FloatTensor([1, 0.1, 0.01]))   
得到的结果就是他们每个分量的梯度---得到的梯度就是他们原本的梯度分别 *乘以 1， 0.1， 0.01

In [15]:
# Compute gradients
y.backward()

In [16]:
print(x.grad)
print(w.grad)
print(b.grad)

tensor([2.])
tensor([1.])
tensor([1.])


In [17]:
b

tensor([3.], requires_grad=True)

In [18]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)
y = x * 2
#y.backward()
#print(x.grad)
y.backward(torch.FloatTensor([1, 0.1, 0.01]))
print(x.grad)

tensor([2.0000, 0.2000, 0.0200])


In [19]:
x

tensor([ 0.0519,  1.8997, -1.8134], requires_grad=True)

# 3.1.3 Dataset   
__torch.utils.data.Dataset__ is abstract class(抽象类).  
定义你的数据类可以 继承和重写这个抽象类，只需要定义 \_\_len\_\_ and \_\_getitem__ two functions.  
通过迭代的方式来取得每一个数据。

In [20]:
import pandas as pd
from torch.utils.data import Dataset
class myDataset(Dataset):
    def __init__(self, csv_file, txt_file, root_dir, other_file):
        self.csv_data = pd.read_csv(csv_file)
        with open(txt_file, 'r') as f:
            data_list = f.readlines()
        self.txt_data = data_list
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.csv_data)
    
    def __getitem__(self, idx):
        data = (self.csv_data[idx], self.txt_data[idx])
        return data

__torch.utils.data.DataLoader__ 定义一个新的迭代器   
可以取batch,shuffle，或者多线程去读取数据。

In [24]:
class example(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]


In [28]:
from torch.utils.data import DataLoader

data = list(range(10))
mydata = example(data)
loader = DataLoader(mydata, batch_size=4)  # each row, we have batch-size is 4
for batch in loader:
    print(batch)

tensor([0, 1, 2, 3])
tensor([4, 5, 6, 7])
tensor([8, 9])


# 3.1.4 nn.Module

所有层结构和损失函数都来自于torch.nn。
__层结构__: 线性层nn.Linear, 1维convolutional layer，2d convolutional layer  
__损失函数__: nn.CrossEntropyLoss()

# 3.1.5 torch.optim

torch.optim.SGD：随机梯度下降（stochastic gradient descent）
torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 3.1.6 Save and load the model

1. save all constructures and parameters of the whole model. Therefore, save model.  
torch.save(model, './model.path')  
torch.load('./model.path')  

2. save parameters of the model. Therefore, save the state of the model.  
torch.save(model.state_dict(0, './model_state.path')  
state = torch.load('model_state.path')  
model.load_state_dic(state)
